# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846886452536                   -0.70    4.5
  2   -7.852325906724       -2.26       -1.53    1.0
  3   -7.852645950529       -3.49       -2.53    3.0
  4   -7.852646676766       -6.14       -3.36    2.2
  5   -7.852646686106       -8.03       -4.76    1.5
  6   -7.852646686726       -9.21       -5.27    4.5
  7   -7.852646686730      -11.48       -5.92    2.0
  8   -7.852646686730      -12.60       -7.56    1.5
  9   -7.852646686730      -14.21       -8.17    2.5
 10   -7.852646686730   +  -14.75       -8.52    1.0
 11   -7.852646686730   +  -14.57       -9.51    2.8
 12   -7.852646686730      -14.27       -9.00    1.0
 13   -7.852646686730   +  -14.21       -8.44    2.0
 14   -7.852646686730      -14.57       -8.75    1.5
 15   -7.852646686730   +  -15.05       -9.15    1.5
 16   -7.852646686730   +  -15.05       -9.46    1.0
 17   -7.852646686730      -14.75      -10.14 

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846852630457                   -0.70           4.8
  2   -7.852526385640       -2.25       -1.64   0.80    2.0
  3   -7.852636641666       -3.96       -2.72   0.80    1.0
  4   -7.852646469800       -5.01       -3.28   0.80    2.2
  5   -7.852646682615       -6.67       -4.14   0.80    1.5
  6   -7.852646686393       -8.42       -4.84   0.80    1.2
  7   -7.852646686726       -9.48       -5.83   0.80    1.8
  8   -7.852646686730      -11.38       -6.86   0.80    2.2
  9   -7.852646686730      -12.84       -7.68   0.80    2.0
 10   -7.852646686730   +  -14.75       -8.38   0.80    2.0
 11   -7.852646686730      -14.75       -8.96   0.80    1.8
 12   -7.852646686730      -14.75       -9.44   0.80    2.0
 13   -7.852646686730   +  -14.75      -10.22   0.80    1.2
 14   -7.852646686730   +    -Inf      -11.62   0.80    2.0
 15   -7.852646686730   +    -Inf      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.370031e+01     3.507393e+00
 * time: 0.47321081161499023
     1     1.229991e+00     2.045906e+00
 * time: 0.7202389240264893
     2    -1.461390e+00     2.427145e+00
 * time: 0.7483489513397217
     3    -3.619530e+00     2.187833e+00
 * time: 0.7883598804473877
     4    -4.747730e+00     1.993412e+00
 * time: 0.8284928798675537
     5    -6.597552e+00     1.470830e+00
 * time: 0.8683428764343262
     6    -7.325277e+00     7.427152e-01
 * time: 0.9085009098052979
     7    -7.598429e+00     5.225700e-01
 * time: 1.0333869457244873
     8    -7.726718e+00     1.433323e-01
 * time: 1.0610589981079102
     9    -7.788030e+00     3.179562e-01
 * time: 1.0888149738311768
    10    -7.823120e+00     7.060991e-02
 * time: 1.1166977882385254
    11    -7.842499e+00     5.458105e-02
 * time: 1.1445269584655762
    12    -7.849838e+00     4.094358e-02
 * time: 1.1723098754882812
    13    -7.851574e+00     1.890222e-02
 * time: 1.200105905

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846750853011                   -0.70    4.5
  2   -7.852289617759       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686386                   -2.55
  2   -7.852646686730       -9.46       -5.67
  3   -7.852646686730   +  -15.05      -10.81
  4   -7.852646686730   +    -Inf      -15.98


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 4.094177364396816e-14
|ρ_newton - ρ_scfv| = 3.2131371196960597e-13
|ρ_newton - ρ_dm|   = 1.563901075306575e-9
